# 17.1 示例: 网络下载的三种风格

## 17.1.1 依序下载的脚本

In [2]:
# BEGIN FLAGS_PY 依序下载
import os
import time
import sys

import requests  # 导入 requests 库。

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # 列出人口最多的 20 个国家的国家代码，按照人口数量排序

BASE_URL = 'http://flupy.org/data/flags'  # 获取国旗图像的网站

DEST_DIR = 'downloads/'  # 保存图像的本地目录


def save_flag(img, filename):  # 下载 img
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):  # 指定国家代码，构建 URL，然后下载图像，返回向应中的二进制内容
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):  # 显示一个字符串，然后刷新 sys.stdout 缓冲，这样能在一行消息中看到进度。在 Python 中得这么做，因为正常情况下，遇到换行才会刷新缓冲
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):  # download_many 是与并发实现比较的关键函数
    for cc in sorted(cc_list):  # 按照字母表迭代国家代码列表，明确表明输出的顺序与输入一直。返回下载的国旗数量
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')

    return len(cc_list)


def main(download_many):  # main 函数记录并报告运行 download_many 的耗时
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))


if __name__ == '__main__':
    # main 函数必须调用执行下载的函数，我们把 download_many 函数当作参数传给 main 函数
    # 这样 main 函数可以用作库函数，在后面的示例中接收 download_many 函数的其他实现
    main(download_many)  
# END FLAGS_PY

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 33.33s


## 17.1.2 使用 concurrent.futures 模块下载

concurrent.futures 模块的主要特色是 ThreadPoolExecutor 和 ProcessPoolExecutor 类，这两个类实现的接口能分别在不同的线程和进程中执行可调用的对象。这两个类在内部维护着一个工作线程或进程池，以及需要执行的任务队列。不过，这个接口抽象的层级很高，像下载国旗这种案例，无需关心任何实现细节

下面例子展示如何使用 ThreadPoolExecutor.map 方法，已最简单的方式实现并发下载

In [4]:
# BEGIN FLAGS_THREADPOOL
from concurrent import futures

# from flags import save_flag, get_flag, show, main  # <1>

MAX_WORKERS = 20  # 设定 ThreadPoolExecutor 类最多使用几个线程


def download_one(cc):  # 下载一个图像的函数，这是在各个线程中执行的函数
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))  # 设定工作线程数量，使用允许的最大值与要处理数量间较小的那个值，以免创建多余的线程
    with futures.ThreadPoolExecutor(workers) as executor:  
        # 使用工作的线程数实例化 ThreadPoolExecutor 类；executor.__exit__ 方法会调用 executor.shutdown(wait = True)方法，它会在所有线程都执行完毕前阻塞线程

        # map 方法的作用与内置的 map 函数相似，不过 download_one 函数会在多个线程中并发调用
        # map 方法返回一个生成器，因此可以迭代，获取各个函数返回的值
        res = executor.map(download_one, sorted(cc_list)) 

    return len(list(res))  # <7>


if __name__ == '__main__':
    main(download_many)  # <8>
# END FLAGS_THREADPOOL


EG DE ID NG PH IR US PK MX BD VN JP FR ET CN IN TR RU BR CD 
20 flags downloaded in 4.34s


##  17.1.3 future 在哪里

future 是 concurrent.futures 模块和 asyncio 包的重要组件，可是作为这两个库的用户，我们有时却见不到 future。

从 Python 3.4 起，标准库中有两个名为 Future 的类: concurrent.futures.Future 和 asyncio.Future。
这两个类的作用相同: 两个 Future 类的实例都表示可能已经完成或者尚未完成的延迟计算

future 封装待完成的操作，可以放入队列，完成的状态可以查询，得到结果(或抛出异常)后可以获取结果(或异常)

我们要记住一件事：通常情况下自己不应该创建 future，而只能由并发框架实例化。
原因很简单: future 表示终将发生的事情，而确定某件事发生的唯一方式是执行的事件已经排定。因此只有排定把某件事交给 concurrent.futures.Executor 子类处理时，才会创建 concurrent.futures.Future 实例。
例如: Executor.submit() 方法的参数是一个可调用的对象，调用这个方法后会为传入的可调用对象排期，并返回一个 future。

客户端代码不应该改变 future 的状态，并发框架在 future 表示的延迟计算结束后会改变 future 的状态，而我们无法控制计算何时结束。

这两种 future 都有 .done() 方法，这个方法不阻塞，返回的是布尔值，指明 future 链接的可调用对象是否已经执行。客户端代码通常不会询问 future 是否运行结束，而是会等待通知。
因此，两个 Future 类都有 .add_done_callback() 方法: 这个方法只有一个参数，类型是可调用的对象，future 运行结束后会调用指定的可调用对象。

此外，还有 .result() 方法。在 future 运行结束后调用的话，这个方法在两个 Future 类中的作用相同: 返回可调用对象的结果，或重新抛出执行可调用对象时抛出的异常。
可是，如果 future 没有运行结束，result 方法在两个 Future 类中的行为相差很大。
对 concurrent.futures.Future 实例来说，调用 f.result() 方法会阻塞调用方所在的线程，直到结果有所返回。此时，result 方法可以接收可选的 timeout 参数，如果在指定的时间内 future 没有运行完毕，会抛出 TimeoutError 异常。但 asyncio.Future.result 方法不支持设定超过时间，在那个库中获取 future 的结果最好使用 yield from 结构。

这两个库中有几个函数会返回 future，其他函数则使用 future，以用户易于理解的方式实现自身。Executor.map 方法属于后者: 返回值是一个迭代器，迭代器的 \_\_next__ 方法调用各个 future 的 result 方法，因此我们得到的是各个 future 的结构，而非 future 本身。

为了从实用的角度理解 future，我们可以实用 concurrent.future.as_completed 函数重写上一节的实现，这个函数的参数是一个 future 列表，返回值是一个迭代器，在 future 运行结束后产出 future

为了使用 futures.as_compelted 函数，只需修改 download_many 函数，把较抽象的 executor.map 调用换成两个 for 循环: 一个用于创建并排定 future，另一个用于获取 future 的结构

In [5]:
from concurrent import futures

# from flags import save_flag, get_flag, show, main

MAX_WORKERS = 20


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


# BEGIN FLAGS_THREADPOOL_AS_COMPLETED
def download_many(cc_list):
    cc_list = cc_list[:5]  # 这此演示只使用人口最多的 5 个国家
    with futures.ThreadPoolExecutor(max_workers=3) as executor:  # max_workers 为 3 
        to_do = []
        for cc in sorted(cc_list):  # 按照字母表迭代国家代码
            future = executor.submit(download_one, cc)  # executor.submit() 方法排定可调用对象的执行时间，然后返回一个 future，表示这个待执行的操作
            to_do.append(future)  # 储存各个 future，后面传给 as_completed 函数
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))  # 显示一个消息，包含国家代码和对应的 future

        results = []
        for future in futures.as_completed(to_do):  # as_completed 在 future 运行结束后产出 future
            res = future.result()  # 获取该 future 的结果
            msg = '{} result: {!r}'
            print(msg.format(future, res)) # 显示 future 及其结果
            results.append(res)

    return len(results)
# END FLAGS_THREADPOOL_AS_COMPLETED

if __name__ == '__main__':
    main(download_many)

Scheduled for BR: <Future at 0x1d97c08f6a0 state=running>
Scheduled for CN: <Future at 0x1d97c087640 state=running>
Scheduled for ID: <Future at 0x1d97dd7b370 state=running>
Scheduled for IN: <Future at 0x1d97dd7be20 state=pending>
Scheduled for US: <Future at 0x1d97ddd8fa0 state=pending>
ID <Future at 0x1d97dd7b370 state=finished returned str> result: 'ID'
BR <Future at 0x1d97c08f6a0 state=finished returned str> result: 'BR'
IN <Future at 0x1d97dd7be20 state=finished returned str> result: 'IN'
CN <Future at 0x1d97c087640 state=finished returned str> result: 'CN'
US <Future at 0x1d97ddd8fa0 state=finished returned str> result: 'US'

5 flags downloaded in 3.84s


前三个 future 的状态是 running，因为有三个工作的线程  
后两个 future 的状态是 pending，等待有线程可用

结果行的第一个国家代码是运行在一个工作线程中的 download_one 函数输出的，随后的内容是 download_many 函数输出的

严格来说，我们目前测试的并发脚本都不能并行下载。使用 concurrent.futures 库实现的那两个示例受 GIL(Global Interpreter Lock, 全局解释器锁) 的限制，而 asyncio 脚本在单个线程中运行

# 17.2 阻塞型I/0 和 GIL

CPython 解释器本身就不是线程安全的，因此有 GIL，一次只允许使用一个线程执行 Python 字节码。因此，一个 Python 进程通常不能同时使用多个 CPU 核心。

Python 标准库中单所有阻塞型I/0函数都会释放 GIL，允许其他线程允许。time.sleep() 函数也会释放 GIL。因此，尽管有 GIL，Python 线程还是能在 I/0 密集型应用发挥作用

# 17.3 使用 concurrent.futures 模块启动进程

这个模块可用实现真正的并行计算(绕开 GIL)，因为它使用 ProcessPoolExecutor 类把工作分配给多个 Python 进程处理。因此，如果需要做 CPU 密集型处理，使用这个模块能绕开 GIL，利用所有可用的 CPU 核心。

下载国旗的示例或其他 I/O 密集型作业使用 ProcessPoolExecutor 类得不到任何好处。这一点易于验证，只需要简单修改 download_many 的代码
```python
    def down_many(cc_list):
        workers = min(MAX_WORKERS, len(cc_list))
        with futures.ThreadPoolExecutor(workers) as executor:
```
改成
```python
    def down_many(cc_list):
        with futures.ProcessPoolExecutor() as executor:
```

对于简单用途来说，这两个实现 Executor 接口的类唯一值得注意的区别是，ThreadPoolExecutor.\_\_init__ 方法需要 MAX_WORKERS 参数，指定线程池中线程的数量。
在 ProcessPoolExecutor 类中，那个参数的可选的，而且大多数情况都不使用 —— 默认值是 os.cpu_count() 返回的 CPU 数量。

如果使用 Python 处理 CPU 密集型工作，应该试试 PyPy。

# 17.4 实验 Executor.map 方法

In [6]:
# BEGIN EXECUTOR_MAP
from time import sleep, strftime
from concurrent import futures


def display(*args):  # 把传入的参数打印出来，并在前面加上时间
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)


def loiter(n):  # loiter 函数什么也没有做，只是在开始时显示一个消息，然后休眠 n 秒，最后在结束时再显示一个消息；消息使用制表符缩进，缩进的量由 n 的值确定
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10  # loiter 函数返回 n*10


def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)  # 创建 ThreadPoolExecutor 实例，有三个线程
    results = executor.map(loiter, range(5))  # 把五个任务提交给 executor
    display('results:', results)  # 立即显示调用的 executor.map 方法的结果: 一个生成器
    display('Waiting for individual results:')
    for i, result in enumerate(results):  
        # for 循环中的 enumerate 函数会隐式地调用 next(result)，这个函数又会在表示第一个任务的 _f future 上调用 _f.result() 方法。
        # _f.result() 方法会阻塞，直到 future 运行结束，因此这个循环每次迭代都要等待下一个结果做好准备
        
        display('result {}: {}'.format(i, result))


main()
# END EXECUTOR_MAP

[02:42:50] Script starting.
[02:42:50] loiter(0): doing nothing for 0s...
[02:42:50] loiter(0): done.
[02:42:50] 	loiter(1): doing nothing for 1s...
[02:42:50] 		loiter(2): doing nothing for 2s...
[02:42:50] 			loiter(3): doing nothing for 3s...
[02:42:50] results: <generator object Executor.map.<locals>.result_iterator at 0x000001D97D6EDEB0>
[02:42:50] Waiting for individual results:
[02:42:50] result 0: 0
[02:42:51] 	loiter(1): done.
[02:42:51] 				loiter(4): doing nothing for 4s...[02:42:51]
 result 1: 10
[02:42:52] 		loiter(2): done.
[02:42:52] result 2: 20
[02:42:53] 			loiter(3): done.
[02:42:53] result 3: 30
[02:42:55] 				loiter(4): done.
[02:42:55] result 4: 40


* 第一个线程执行 loiter(0)，因此休眠 0 秒，甚至会在第二个线程开始之前就结束，不过具体情况因人而异  
* loiter(1) 和 loiter(2) 立即开始  
* 然后执行 display('results:', results)，表示 executor.map 返回了一个生成器，不管有多少任务，也不管任务上限是多少，目前不会阻塞
* 进入 for 循环后，执行过程可能阻塞，具体情况取决于传给 loiter 函数的参数: result 生成器的 \_\_next__ 方法必须等待第一个future 运行结束，此时不会阻塞，因为 loiter(0) 在循环之前就结束。注意，这一点之前的所有事件都在同一刻发生  
* 一秒钟之后，loiter(1) 运行完毕，这个线程闲置，可以开始运行 loiter(4)
* 显示 loiter(1) 的结果: 10，现在，for 循环会阻塞，等待 loiter(2) 的结果。
* 同上 loiter(2) 运行完毕，阻塞等待 3
* 2s 后 4 运行结束，因为他从 51 开始，休眠 4s

Executor.map 函数易于使用，但它有一个特性: 这个函数返回结果的顺序与调用开始的顺序一致。

# 17.5 显示下载进度并处理错误

为了处理错误，我们创建了 flags2 系列实例
* flags2_common.py: 这个模块包含所有 flag2 示例通用的函数和设置，但与本章的话题没有直接关系
* flags2_sequential.py: 能正确处理错误，以及显示进度条的 HTTP 依序下载客户端。flags2_threadpool 脚本会用到这个模块的 download_one 函数
* flags2_threadpool.py:基于 futures.ThreadPoolExecutor 类实现的 HTTP 并发客户端，演示如何处理错误，以及集成进度条
* flag2_asyncio.py:与前一个脚本的作用相同，不过使用 asyncio 和 aiohttp 实现

flags2 系列示例最明显的特色是，有使用 TQDM 包实现文本动画进度条。

In [1]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
    time.sleep(.01)

100%|██████████| 1000/1000 [00:15<00:00, 63.37it/s]


flags2 系列示例的另一个特色是，提供了命令行接口，三个脚本接受的选项相同，运行任意一个脚本指定的 -h 选项就能看到所有选项

## 17.5.1 flag2系列示例处理错误的方式

三个脚本在负责下载一个文件的函数 (download_one) 中使用相同的策略处理 HTTP 404 错误。其他异常则向上冒泡

In [ ]:
# 可处理 HTTP404 的 download_one
# BEGIN FLAGS2_BASIC_HTTP_FUNCTIONS
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:  # get_flag 没有处理错误，当 HTTP 码不是 200 时，使用 requests.Reponse.raise_for_status 方法抛出异常
        resp.raise_for_status()
    return resp.content


def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:  # download_one 函数捕获 requests.Reponse.HTTPError 异常，特别处理 404 错误
        res = exc.response
        if res.status_code == 404:

            # 把局部变量 status 设为 HTTPStatus.not_found；HTTPStatus 是 Enum 对象
            # HTTPStatus = Enum('Status', 'ok not_found error') 枚举
            status = HTTPStatus.not_found 
            msg = 'not found'
        else:  # 其他异常则向上冒泡
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'

    if verbose:  # 是否打印参数
        print(cc, msg)

    # Result = namedtuple('Result', 'status data')
    # download_one 的返回值是一个 namedtuple ———— Result，其中有一个字段的值是 HTTPStatus.not_found 或 HTTPStatus.ok
    return Result(status, cc)  
# END FLAGS2_BASIC_HTTP_FUNCTIONS


In [ ]:
# 实现依序下载的 download_many 函数
# BEGIN FLAGS2_DOWNLOAD_MANY_SEQUENTIAL
def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter()  # 这个 Counter 示例用于统计不同的下载状态
    cc_iter = sorted(cc_list)  # 按字母顺序传入的国家代码列表，保存在 cc_iter 中
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter)  # 如果不是详细模式，把 cc_iter 传给 tqdm 函数，返回一个迭代器，产出 cc_iter 的元素，还会显示进度条动画
    for cc in cc_iter:  # for 循环迭代 cc_iter
        try:
            res = download_one(cc, base_url, verbose)  # 调用 download_one，执行下载
        except requests.exceptions.HTTPError as exc:  # 捕获 HTTPError 异常
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msg.format(res=exc.response)
        except requests.exceptions.ConnectionError as exc:  # 处理其他与网络有关的异常。其他异常会中止脚本
            error_msg = 'Connection error'
        else:  # 如果没有异常，则获取 status
            error_msg = ''
            status = res.status

        if error_msg:
            status = HTTPStatus.error  # 如果有错误，把局部变量 status 变成相应状态
        counter[status] += 1  # 记录错误数量
        if verbose and error_msg: # 如果是详细模式且有错误，显示错误信息
            print('*** Error for {}: {}'.format(cc, error_msg))

    return counter  # 返回 counter，以便 main 函数能在最终报告中显示数量
# END FLAGS2_DOWNLOAD_MANY_SEQUENTIAL

## 17.5.2 使用 future.as_completed 函数

In [ ]:
"""Download flags of countries (with error handling).

ThreadPool version

Sample run::

    $ python3 flags2_threadpool.py -s ERROR -e
    ERROR site: http://localhost:8003/flags
    Searching for 676 flags: from AA to ZZ
    30 concurrent connections will be used.
    --------------------
    150 flags downloaded.
    361 not found.
    165 errors.
    Elapsed time: 7.46s

"""

# BEGIN FLAGS2_THREADPOOL
import collections
from concurrent import futures

import requests
import tqdm  # 导入进度条相关库

from flags2_common import main, HTTPStatus  # <2>
from flags2_sequential import download_one  # <3>

DEFAULT_CONCUR_REQ = 30  # 默认的并发最大值
MAX_CONCUR_REQ = 1000  # 限制最大的并发请求数


def download_many(cc_list, base_url, verbose, concur_req):
    counter = collections.Counter()
    with futures.ThreadPoolExecutor(max_workers=concur_req) as executor:  # 把 max_workers 设为 concur_req，创建 ThreadPoolExecutor 实例
        to_do_map = {}  # 这个字典把各个 Future 实例映射到相同的国家代码上
        for cc in sorted(cc_list):  # 按字母表迭代国家代码列表
            future = executor.submit(download_one,
                            cc, base_url, verbose)  # 逐个加入线程池
            to_do_map[future] = cc  # 把 future 和 国家代码储存在字典中
        done_iter = futures.as_completed(to_do_map)  # futures.as_completed 返回迭代器，在 future 运行结束后产出 future
        if not verbose:

            # 如果不是详细模式，把 done_iter 给 tadm 函数，显示进度条，因为 done_iter 没有 len 函数，我们手动设置进度条
            done_iter = tqdm.tqdm(done_iter, total=len(cc_list))  
        for future in done_iter:  # 迭代运行后的 future
            try:
                res = future.result()  # 在 future 上调用 result 方法，返回值或抛出异常
            except requests.exceptions.HTTPError as exc:  # 捕获 HTTPError 异常
                error_msg = 'HTTP {res.status_code} - {res.reason}'
                error_msg = error_msg.format(res=exc.response)
            except requests.exceptions.ConnectionError as exc:
                error_msg = 'Connection error'
            else:
                error_msg = ''
                status = res.status

            if error_msg:
                status = HTTPStatus.error
            counter[status] += 1
            if verbose and error_msg:
                cc = to_do_map[future]  # 为了给错误信息提供上下文，以当前 future 为键，从 to_do_map 中获取国家代码
                print('*** Error for {}: {}'.format(cc, error_msg))

    return counter


if __name__ == '__main__':
    main(download_many, DEFAULT_CONCUR_REQ, MAX_CONCUR_REQ)
# END FLAGS2_THREADPOOL


上面的代码中使用了 futures.as_completed 的惯用法: 构建一个字典，将各个 future 映射到其他数据上。

## 17.5.3 线程和多进程的替代方案

对简单的程序来说，可用使用 multiprocessing 模块代替 threading 模块，少量改动极客。不过 multiprocessing 模块还能解决协作进程遇到的最大挑战: 在进程间传递数据